In [1]:
import json
import io
from pathlib import Path

import pandas as pd

In [2]:
data_path = Path('./data/verdd_lexemes.json')

In [3]:
with io.open(data_path, 'r', encoding='utf-8') as f:
    all_data = json.load(f)

In [4]:
language = "sms"

In [5]:
lang_lexemes = [_data for _data in all_data if _data["model"] == "manageXML.lexeme" and _data["fields"]["language"] == language]

In [6]:
len(lang_lexemes)

28984

In [7]:
lexeme_data = {lexeme["pk"]: lexeme for lexeme in lang_lexemes}

In [8]:
for _data in all_data:
    if _data["model"] == "manageXML.stem":
        _lexeme_id = _data["fields"]["lexeme"]
        if _lexeme_id not in lexeme_data:
            continue
        if "stems" not in lexeme_data[_lexeme_id]["fields"]:
            lexeme_data[_lexeme_id]["fields"]["stems"] = []
        lexeme_data[_lexeme_id]["fields"]["stems"].append(_data["fields"])

In [9]:
data = []
for lexeme_id, lexeme in lexeme_data.items():
    lexeme_fields = lexeme["fields"]
    lexeme_name = lexeme_fields.get("lexeme", "")
    
    if not lexeme_name:
        continue
    
    pos = lexeme_fields.get("pos", "")
    stems = lexeme_fields.get("stems", [])
    
    for stem in stems:
        data.append({
            "lexeme": lexeme_name,
            "language": language,
            "pos": pos,
            "stem_text": stem["text"],
            "contlex": stem["contlex"],
        })

df = pd.DataFrame(data)

output_path = Path('./data/lexeme_stems.csv')
df.to_csv(output_path, index=False)